In [18]:
import zarr
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import random
import os

# USER INPUTS - Change these values
dataset = 'ORION-CRC' ## or Xenium
case = 'CRC01'
modality = "mif"  # or "he"
sample_number = 5  # or 'random' for random selection

# Set paths
base_path = f"/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/{dataset}/zarr_data/{case}"
zarr_path = os.path.join(base_path, modality, "images.zarr")

# Open the zarr array
z = zarr.open(zarr_path, mode='r')

# Get available chunks
chunks = [d for d in os.listdir(zarr_path) if d.endswith('.0.0.0')]
available_samples = sorted([int(d.split('.')[0]) for d in chunks])
print(f"Available samples: {available_samples}")

# Determine chunk index
if sample_number == 'random':
    chunk_idx = random.choice(available_samples)
    print(f"Randomly selected sample: {chunk_idx}")
else:
    chunk_idx = sample_number

print(f"Converting chunk {chunk_idx} to PNG...")

# Read the data
data = z[chunk_idx]

# Create output directory
output_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/"
os.makedirs(output_dir, exist_ok=True)

if modality == "he":
    # Handle HE images (3-channel RGB)
    if data.dtype == np.float32 or data.dtype == np.float64:
        data = ((data - data.min()) / (data.max() - data.min()) * 255).astype(np.uint8)
    elif data.dtype != np.uint8:
        data = data.astype(np.uint8)
    
    if len(data.shape) == 3 and data.shape[2] == 3:
        img = Image.fromarray(data, mode='RGB')
    elif len(data.shape) == 2:
        img = Image.fromarray(data, mode='L')
    else:
        img = Image.fromarray(data[:, :, 0], mode='L')
    
    output_path = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}.png")
    img.save(output_path)
    print(f"Saved to {output_path}")

elif modality == "mif":
    # Handle MIF images (2-channel)
    print(f"MIF image shape: {data.shape}")
    
    # Extract channels
    mif_ch0 = data[:, :, 0]  # Nuclei channel
    mif_ch1 = data[:, :, 1]  # Second channel
    
    # Normalize each channel to 0-255
    def normalize_channel(channel):
        ch_min, ch_max = channel.min(), channel.max()
        if ch_max > ch_min:
            return ((channel - ch_min) / (ch_max - ch_min) * 255).astype(np.uint8)
        return np.zeros_like(channel, dtype=np.uint8)
    
    mif_ch0_norm = normalize_channel(mif_ch0)
    mif_ch1_norm = normalize_channel(mif_ch1)
    
    # Create RGB image: Channel 0 (blue), Channel 1 (green/cyan)
    # R: Channel 1, G: Channel 1, B: Channel 0
    rgb_image = np.zeros((data.shape[0], data.shape[1], 3), dtype=np.uint8)
    rgb_image[:, :, 0] = mif_ch1_norm  # Red channel = Channel 1
    rgb_image[:, :, 1] = mif_ch1_norm  # Green channel = Channel 1 (makes it cyan)
    rgb_image[:, :, 2] = mif_ch0_norm  # Blue channel = Channel 0
    
    # Save combined RGB image
    img_combined = Image.fromarray(rgb_image, mode='RGB')
    output_path_combined = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_combined.png")
    img_combined.save(output_path_combined)
    print(f"Saved combined RGB image to {output_path_combined}")
    
    # Also save individual channel visualizations for reference
    # Channel 0 (hot colormap)
    plt.figure(figsize=(10, 10))
    plt.imshow(mif_ch0, cmap='hot')
    plt.axis('off')
    plt.title('MIF Channel 0 (Nuclei)', fontsize=14)
    output_path_ch0 = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_ch0.png")
    plt.savefig(output_path_ch0, bbox_inches='tight', pad_inches=0, dpi=150)
    plt.close()
    print(f"Saved Channel 0 to {output_path_ch0}")
    
    # Channel 1 (viridis colormap)
    plt.figure(figsize=(10, 10))
    plt.imshow(mif_ch1, cmap='viridis')
    plt.axis('off')
    plt.title('MIF Channel 1', fontsize=14)
    output_path_ch1 = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_ch1.png")
    plt.savefig(output_path_ch1, bbox_inches='tight', pad_inches=0, dpi=150)
    plt.close()
    print(f"Saved Channel 1 to {output_path_ch1}")
    
    # Save side-by-side comparison
    fig, axes = plt.subplots(1, 3, figsize=(30, 10))
    
    axes[0].imshow(mif_ch0, cmap='Blues')
    axes[0].set_title('Channel 0 (Nuclei - Blue)', fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(mif_ch1, cmap='Greens')
    axes[1].set_title('Channel 1 (Cyan)', fontsize=14, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(rgb_image)
    axes[2].set_title('Combined (Ch0=Blue, Ch1=Cyan)', fontsize=14, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    output_path_comparison = os.path.join(output_dir, f"{case}-{modality}_chunk_{chunk_idx}_comparison.png")
    plt.savefig(output_path_comparison, bbox_inches='tight', dpi=150)
    plt.close()
    print(f"Saved comparison to {output_path_comparison}")

Available samples: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
Converting chunk 5 to PNG...
MIF image shape: (512, 512, 2)
Saved combined RGB image to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_combined.png
Saved Channel 0 to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_ch0.png
Saved Channel 1 to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_ch1.png
Saved comparison to /rsrch9/home/plm/idso_fa1_pathology/TIER2/yasin-vitaminp/png/CRC01-mif_chunk_5_comparison.png
